### SET PARAMETERS

In [ ]:
from pathlib import Path

# DATA SETTINGS
TRAIN_PATH = Path('data/vids/split/train')
TEST_PATH = Path('data/vids/split/test')
FRAMES_PER_VIDEO = 10
IMG_SIZE = 256

# TRAINING SETTINGS
MODEL_NAME = "MAX"
EPOCHS = 1
BATCH_SIZE = 2
OPTIMIZER = "adam"
PRETRAINED = False

### LOAD DATA

In [ ]:
import tensorflow as tf
from src.datagen import FrameGenerator

# CREATE TRAINING DATASET
output_signature = (tf.TensorSpec(shape = (FRAMES_PER_VIDEO, IMG_SIZE, IMG_SIZE, 3), 
                                  dtype = tf.float32), tf.TensorSpec(shape = (1), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(TRAIN_PATH, FRAMES_PER_VIDEO, (IMG_SIZE, IMG_SIZE), training=True),
                                          output_signature = output_signature)

# CREATE TESTING DATASET
test_ds = tf.data.Dataset.from_generator(FrameGenerator(TEST_PATH, FRAMES_PER_VIDEO, (IMG_SIZE, IMG_SIZE), training=False),output_signature = output_signature)

# ADD BATCH DIMENSION (IF USING TIME DIM)
train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

# VERIFY DATA HAS CORRECT SHAPE
train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

test_frames, test_labels = next(iter(test_ds))
print(f'Shape of validation set of frames: {test_frames.shape}')
print(f'Shape of validation labels: {test_labels.shape}')

### INITIALIZE MODEL

In [ ]:
from src.models import get_mouthnet_model, get_mesonet_model
from src.custom_model_max import get_cnn_rnn_hybrid
from src.train import load_model_from_path

if not PRETRAINED:
    # LOAD NEW MODEL
    model = get_cnn_rnn_hybrid()
else:
    # LOAD MODEL WEIGHTS FROM FILE
    model = load_model_from_path(f"saved_models/{MODEL_NAME}.keras")
    model.summary()

### TRAIN MODEL
Will automatically save model (under saved_models/MODEL_NAME.keras) whenever new maximum validation accuracy reached.

In [ ]:
from src.train import prep_and_train_model, save_history
from time import perf_counter

start = perf_counter()

history, model = prep_and_train_model(
    model=model,
    optim=OPTIMIZER,
    train_data=train_ds,
    test_data=test_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    model_name=MODEL_NAME
)

print(f"TOTAL TRAIN TIME: {perf_counter() - start}")

# SAVE HISTORY TO FILE
save_history(history, MODEL_NAME)

### VISUALIZE TRAINING HISTORY

In [ ]:
import matplotlib.pyplot as plt

# NOTE: ONLY WORKS FOR MULTIPLE EPOCHS

# PLOT ACCURACY
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# PLOT LOSS
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()